In [1]:
from instruct_ReID import get_model,extract_feature_from_model
ins_ReID=get_model().eval()

c:\Users\hthek\Downloads\mct\MCDPT\instruct_ReID.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(checkpoint_path, map_location='cpu')


Detected 751 classes from checkpoint


c:\Users\hthek\anaconda3\envs\tracking\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\Users\hthek\anaconda3\envs\tracking\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


using stride: [16, 16], and patch number is num_y16 * num_x8


c:\Users\hthek\anaconda3\envs\tracking\lib\site-packages\transformers\modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file,

Successfully loaded checkpoint weights


In [1]:
import cv2
import torch
import torch.nn as nn
import numpy as np
from pathlib import Path
from PIL import Image
from boxmot import BoostTrack, BotSort, StrongSort
from torchvision.models.detection import (
    fasterrcnn_resnet50_fpn_v2,
    FasterRCNN_ResNet50_FPN_V2_Weights as Weights
)
from mc_tracker import sct,mct
from ultralytics import YOLO
from torchvision import transforms



In [25]:
class Instruct_ReID(nn.Module):
    def __init__(self, model, mode="sc"):
        
        super().__init__()
        self.model=model.cuda()
        self.mode=mode
        self.data_transform=transforms.Compose([
                              transforms.ToPILImage(),
                              transforms.Resize((256, 128)),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                              ])
    def forward(self,input):
        if self.mode=="sc":
            text="Do not change clothes"

        insreid_emb = torch.cat([self.model(self.data_transform(image).unsqueeze(0).cuda(),text, this_task_info=None)[3] for image in input],dim=0).cpu()
        return insreid_emb



In [26]:
class OSNet(nn.Module):
    def __init__(self, model, mode="sc"):
        
        super().__init__()
        self.model=model.cuda()
        self.data_transform=transforms.Compose([
                              transforms.ToPILImage(),
                              transforms.Resize((256, 128)),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                              ])
    def forward(self,input):
       

        # osnet_emb = torch.cat([self.model(self.data_transform(image).unsqueeze(0).cuda()) for image in input],dim=0).cpu()
        batch = torch.cat([self.data_transform(image).unsqueeze(0).cuda() for image in input],dim=0)
        print(batch.shape)
        osnet_emb = self.model(batch.cuda()).cpu()
        return osnet_emb




In [4]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)
client

In [39]:

reid_model=OSNet(model=osnet).cuda()
tracker = sct.SingleCameraTracker(cam_id=0,reid_model=reid_model)

In [40]:
def get_screen_edges(detections, frame_shape, edge_margin=20):
    H,W = frame_shape[:2]
    edge_flages=[]
    for (x1,y1,x2,y2) in detections[:,:4]:
        near_edge=(x1 <= edge_margin or y1 <= edge_margin            
                   or W-x2 <= edge_margin or H-y2 <= edge_margin) 
        edge_flages.append(near_edge)
    return edge_flages


In [ ]:
# Load detector with pretrained weights and preprocessing transforms
from torchvision import transforms
import matplotlib.pyplot as plt
data_transform=transforms.Compose([
                              transforms.ToPILImage(),
                              transforms.Resize((256, 128)),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                              ])

det_across_frames=[]
def crop_frame(frame, box):
    x1,y1,x2,y2=box.astype(np.int32)

    cropped = frame[y1:y2,x1:x2]
    return cropped

def process_crops(crops, transform):
    transformed_crops = torch.stack([transform(crop) for crop in crops])
    final_crops = transformed_crops.unsqueeze(2).repeat(1,1,8,1,1)
    return final_crops
# writer = cv2.VideoWriter("osnet_x0_25_msmt17.mp4",cv2.VideoWriter_fourcc(*"mpv4"),25,(1440,710))
weights = Weights.DEFAULT
# detector = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.5)
# detector.to(device).eval()
# transform = weights.transforms()

detector = YOLO(r"rtdetr-l.pt")
detector.to(device).eval()
transform = weights.transforms()
# vslaclip_reid = torch.load("vslaclip.pth").to(device).eval()

# Initialize tracker
# tracker = BotSort(reid_weights=Path('clip_market1501.pt'), device=device, half=False)


# Start video capture
video_path = r'C:\Users\hthek\OneDrive\Desktop\Collected Dataset\Compressed_Vids\Vid_1.mp4'
cap = cv2.VideoCapture(video_path)
count=0
with torch.inference_mode():
    while True:
        
        success, frame = cap.read()
        if not success:
            break

        # Convert frame to RGB and prepare for detector
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
   

        # Run detection
        if count%1==0:
            output = detector(frame)[0].boxes
            labels = output.cls.cpu().numpy()
            only_people = labels == 0
            scores = output.conf.cpu().numpy()
            scores = scores[only_people]
            keep = scores >= 0.25
            filtered_scores = scores[keep]
            

        
        
        # Prepare detections for tracking
            boxes = output.xyxy.cpu().numpy()[only_people][keep]
            # labels = labels[only_people][keep]
            # detections = np.concatenate([boxes, filtered_scores[:, None], labels[:, None]], axis=1)
        
       
            
        
          
            
        
       
            
            
            
        

        # Update tracker and draw results
        #   INPUT:  M X (x, y, x, y, conf, cls)
        #   OUTPUT: M X (x, y, x, y, id, conf, cls, ind)
        H, W, _ = frame.shape
        boundary_coord = [0, 0, W, H]
        screen_edges = get_screen_edges(boxes,frame.shape)
        boxes=boxes.astype(np.int32)
        tracker.process(frame,boxes,screen_edges,boundary_coord)
        for obj in tracker.get_tracked_objects():
            if obj.display:
                x1,y1,x2,y2 = obj.rect
                id = obj.label.split("-")[-1]
                int_id = int(id)
                color = (max(10*int_id,255),max(100+int_id,255),min(255-5*int_id,0))
                print(id)
                cv2.rectangle(frame,(x1,y1),(x2,y2),color=color,thickness=2)
                cv2.putText(frame,"id: "+id,(x1,y1-5),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=0.5,color=color)
        
        

        # writer.write(frame)
        # if count>5:
        #     break
        # count+=1
        
       

        
        
        cv2.imshow('BoXMOT + Torchvision', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        

# Clean up
cap.release()
cv2.destroyAllWindows()


0: 640x640 22 persons, 1 tie, 3 clocks, 65.5ms
Speed: 2.8ms preprocess, 65.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
torch.Size([22, 3, 256, 128])
4
5
6
7
9
10
13
15
16
18
19
21
24
26
28
31
32
62
267

0: 640x640 22 persons, 1 bench, 1 tie, 3 clocks, 53.6ms
Speed: 5.6ms preprocess, 53.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
torch.Size([22, 3, 256, 128])
4
5
6
7
9
10
13
15
16
18
19
21
24
26
28
31
32
62
267

0: 640x640 21 persons, 1 bench, 1 tie, 3 clocks, 53.0ms
Speed: 4.2ms preprocess, 53.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
torch.Size([21, 3, 256, 128])
4
5
6
7
9
10
13
15
16
18
19
21
24
26
28
31
32
62
145
267

0: 640x640 23 persons, 1 bench, 1 handbag, 1 tie, 4 clocks, 56.0ms
Speed: 4.5ms preprocess, 56.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
torch.Size([23, 3, 256, 128])
4
5
6
7
9
10
13
15
16
18
19
21
24
26
28
31
32
62
145
267

0: 640x640 23 persons, 1 bench, 1 handbag

: 

## MCT

In [ ]:
# Load detector with pretrained weights and preprocessing transforms
from torchvision import transforms
import matplotlib.pyplot as plt


sct_config = {
    'match_threshold': 0.25,
    'n_clusters': 4,
    'clust_init_dis_thresh': 0.1,
    'time_window': 10
}


data_transform=transforms.Compose([
                              transforms.ToPILImage(),
                              transforms.Resize((256, 128)),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                              ])

det_across_frames=[]
def crop_frame(frame, box):
    x1,y1,x2,y2=box.astype(np.int32)

    cropped = frame[y1:y2,x1:x2]
    return cropped

def process_crops(crops, transform):
    transformed_crops = torch.stack([transform(crop) for crop in crops])
    final_crops = transformed_crops.unsqueeze(2).repeat(1,1,8,1,1)
    return final_crops
# writer = cv2.VideoWriter("osnet_x0_25_msmt17.mp4",cv2.VideoWriter_fourcc(*"mpv4"),25,(1440,710))
weights = Weights.DEFAULT
# detector = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.5)
# detector.to(device).eval()
# transform = weights.transforms()

detector = YOLO(r"rtdetr-l.pt")
detector.to(device).eval()
transform = weights.transforms()
# vslaclip_reid = torch.load("vslaclip.pth").to(device).eval()

# Initialize tracker
# tracker = BotSort(reid_weights=Path('clip_market1501.pt'), device=device, half=False)


# Start video capture
reid_model=Instruct_ReID(model=ins_ReID).cuda()
mct1=mct.MultiCameraTracker(reid_model=reid_model,cam_id=0,sct_config=sct_config)
mct2=mct.MultiCameraTracker(reid_model=reid_model,cam_id=1,sct_config=sct_config)
video1_path = r'C:\Users\hthek\OneDrive\Desktop\Collected Dataset\Compressed_Vids\Vid_1.mp4'
video2_path = r'C:\Users\hthek\OneDrive\Desktop\Collected Dataset\Compressed_Vids\Vid_2.mp4'
cap = cv2.VideoCapture(video_path)
count=0
with torch.inference_mode():
    while True:
        
        success, frame = cap.read()
        if not success:
            break

        # Convert frame to RGB and prepare for detector
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
   

        # Run detection
        if count%1==0:
            output = detector(frame)[0].boxes
            labels = output.cls.cpu().numpy()
            only_people = labels == 0
            scores = output.conf.cpu().numpy()
            scores = scores[only_people]
            keep = scores >= 0.25
            filtered_scores = scores[keep]
            

        
        
        # Prepare detections for tracking
            boxes = output.xyxy.cpu().numpy()[only_people][keep]
            # labels = labels[only_people][keep]
            # detections = np.concatenate([boxes, filtered_scores[:, None], labels[:, None]], axis=1)
        
       
            
        
          
            
        
       
            
            
            
        

        # Update tracker and draw results
        #   INPUT:  M X (x, y, x, y, conf, cls)
        #   OUTPUT: M X (x, y, x, y, id, conf, cls, ind)
        H, W, _ = frame.shape
        boundary_coord = [0, 0, W, H]
        screen_edges = get_screen_edges(boxes,frame.shape)
        boxes=boxes.astype(np.int32)
        tracker.process(frame,boxes,screen_edges,boundary_coord)
        for obj in tracker.get_tracked_objects():
            x1,y1,x2,y2 = obj.rect
            cv2.rectangle(frame,(x1,y1),(x2,y2),color=(0,0,255),thickness=2)
        
        

        # writer.write(frame)
        # if count>5:
        #     break
        # count+=1
        
       

        
        
        cv2.imshow('BoXMOT + Torchvision', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        

# Clean up
cap.release()
cv2.destroyAllWindows()

In [82]:
for i in tracker.get_tracked_objects():
    print(i)

TrackedObj(rect=array([829, 572, 912, 709], dtype=int32), label='0-1', display=True)
TrackedObj(rect=array([669, 479, 695, 553], dtype=int32), label='0-2', display=True)
TrackedObj(rect=array([310, 363, 361, 467], dtype=int32), label='0-3', display=True)
TrackedObj(rect=array([127, 372, 165, 462], dtype=int32), label='0-4', display=True)
TrackedObj(rect=array([366, 376, 394, 459], dtype=int32), label='0-5', display=True)
TrackedObj(rect=array([172, 355, 217, 451], dtype=int32), label='0-6', display=True)
TrackedObj(rect=array([828, 423, 870, 541], dtype=int32), label='0-7', display=True)
TrackedObj(rect=array([205, 380, 271, 474], dtype=int32), label='0-8', display=True)
TrackedObj(rect=array([735, 435, 770, 518], dtype=int32), label='0-9', display=True)
TrackedObj(rect=array([401, 380, 439, 475], dtype=int32), label='0-10', display=True)
TrackedObj(rect=array([759, 344, 794, 415], dtype=int32), label='0-11', display=True)
TrackedObj(rect=array([679, 323, 710, 407], dtype=int32), label